<a href="https://colab.research.google.com/github/ryanpereirax/Intrusion-detector-with-supervised-models/blob/main/DoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
dos_df = pd.read_csv('/content/DoS_dataset.csv')


In [ ]:
# Amostras de 1% do Dataset Original
amostras_R = dos_df[dos_df['R'] == 'R'].sample(n=5_000, random_state=42)
amostras_T = dos_df[dos_df['R'] == 'T'].sample(n=4_000, random_state=42)
dos_df_balanceado = pd.concat([amostras_R, amostras_T]).sample(frac=1, random_state=42).reset_index(drop=True)

dos_df_balanceado.to_csv('dos_df_proporcional_ajustado.csv', index=False)

In [ ]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 3.9 MB/s eta 0:00:00


In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Carregar o arquivo para examinar os dados
file_path = "/content/dos_df_proporcional_ajustado.csv"
data = pd.read_csv(file_path)

# Verificar as primeiras linhas do dataset para compreender sua estrutura
data.head()


,1478198376.389427,0316,8,05,21,68,09,21.1,21.2,00,6f,R
0,1.478199e+09,0000,8.0,00,00,00,00,00,00,00,00,T
1,1.478199e+09,0260,8.0,18,21,21,30,08,8f,6c,3c,R
2,1.478199e+09,0329,8.0,0c,bc,7f,14,11,20,00,14,R
3,1.478199e+09,0131,8.0,e5,7f,00,00,52,7f,07,27,R
4,1.478199e+09,0000,8.0,00,00,00,00,00,00,00,00,T


In [ ]:
# Renomear as colunas para melhor compreensão
data.columns = ['Timestamp', 'CAN_ID', 'DLC', 'DATA_0', 'DATA_1', 'DATA_2',
                'DATA_3', 'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7', 'Flag']

# Verificar se existem valores ausentes
missing_data = data.isnull().sum()

# Analisar a proporção de flags 'T' e 'R'
flag_distribution = data['Flag'].value_counts(normalize=True)

# Examinar estatísticas descritivas para colunas numéricas
numeric_stats = data.describe()

missing_data, flag_distribution, numeric_stats


(Timestamp    0
 CAN_ID       0
 DLC          0
 DATA_0       0
 DATA_1       0
 DATA_2       0
 DATA_3       0
 DATA_4       0
 DATA_5       0
 DATA_6       0
 DATA_7       0
 Flag         0
 dtype: int64,
 Flag
 R    0.555556
 T    0.444444
 Name: proportion, dtype: float64,
           Timestamp     DLC
 count  9.000000e+03  9000.0
 mean   1.478199e+09     8.0
 std    3.502571e+02     0.0
 min    1.478198e+09     8.0
 25%    1.478199e+09     8.0
 50%    1.478199e+09     8.0
 75%    1.478199e+09     8.0
 max    1.478200e+09     8.0)

In [ ]:
# Converter o Timestamp para um formato de data legível para análise temporal
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='s')

# Criar uma nova coluna para armazenar apenas a hora das mensagens
data['Hour'] = data['Timestamp'].dt.hour

# Contar o número de mensagens por hora
messages_per_hour = data.groupby('Hour').size()

# Identificar quais CAN_IDs são mais frequentes
can_id_distribution = data['CAN_ID'].value_counts().head(10)

messages_per_hour, can_id_distribution


(Hour
 18    8996
 19       4
 dtype: int64,
 CAN_ID
 0000    4000
 0370     295
 02c0     295
 0440     288
 043f     286
 0130     280
 0140     278
 02a0     275
 0316     274
 0260     271
 Name: count, dtype: int64)

In [ ]:
# Verificar a distribuição dos CAN_IDs em relação às flags 'T' e 'R'
can_id_flag_distribution = data.groupby(['CAN_ID', 'Flag']).size().unstack(fill_value=0)

# Explorar padrões nos campos 'DATA_X' para mensagens com flag 'T'
data_injected = data[data['Flag'] == 'T']
data_normal = data[data['Flag'] == 'R']

# Calcular estatísticas descritivas para os campos 'DATA_X' para cada tipo de flag
data_injected_stats = data_injected[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                     'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].describe()

data_normal_stats = data_normal[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                 'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].describe()

can_id_flag_distribution, data_injected_stats, data_normal_stats


(Flag      R     T
 CAN_ID           
 0000      0  4000
 0002    257     0
 00a0     34     0
 00a1     33     0
 0130    280     0
 0131    262     0
 0140    278     0
 0153    241     0
 018f    253     0
 01f1    136     0
 0260    271     0
 02a0    275     0
 02c0    295     0
 0316    274     0
 0329    257     0
 0350    270     0
 0370    295     0
 0430    149     0
 043f    286     0
 0440    288     0
 04b1    140     0
 04f0    137     0
 0545    262     0
 05a0      1     0
 05a2      1     0
 0690     25     0,
        DATA_0 DATA_1 DATA_2 DATA_3 DATA_4 DATA_5 DATA_6 DATA_7
 count    4000   4000   4000   4000   4000   4000   4000   4000
 unique      1      1      1      1      1      1      1      1
 top        00     00     00     00     00     00     00     00
 freq     4000   4000   4000   4000   4000   4000   4000   4000,
        DATA_0 DATA_1 DATA_2 DATA_3 DATA_4 DATA_5 DATA_6 DATA_7
 count    5000   5000   5000   5000   5000   5000   5000   5000
 unique     70    

In [ ]:
# Verificar a distribuição temporal das mensagens com CAN_ID '0000' (flag 'T')
injected_id_0000 = data[(data['CAN_ID'] == '0000') & (data['Flag'] == 'T')]

# Contagem de mensagens injetadas por minuto
injected_id_0000['Minute'] = injected_id_0000['Timestamp'].dt.minute
injected_per_minute = injected_id_0000.groupby('Minute').size()

# Explorar CAN_IDs que aparecem menos frequentemente para identificar possíveis outliers
rare_can_ids = data['CAN_ID'].value_counts().tail(10)

# Verificar se existem correlações entre os campos 'DATA_X' para as mensagens normais
data_normal_numeric = data_normal[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                                   'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].apply(lambda x: x.apply(int, base=16))
correlation_matrix = data_normal_numeric.corr()

injected_per_minute, rare_can_ids, correlation_matrix


<ipython-input-12-8fa582d780b2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injected_id_0000['Minute'] = injected_id_0000['Timestamp'].dt.minute


(Minute
 0       4
 39     98
 40    219
 41    333
 42    144
 43    140
 44    137
 45    174
 46    137
 47    192
 48    260
 49    233
 50    296
 51    151
 52    231
 53    260
 54    181
 55    126
 56    166
 57    159
 58    205
 59    154
 dtype: int64,
 CAN_ID
 0153    241
 0430    149
 04b1    140
 04f0    137
 01f1    136
 00a0     34
 00a1     33
 0690     25
 05a2      1
 05a0      1
 Name: count, dtype: int64,
           DATA_0    DATA_1    DATA_2    DATA_3    DATA_4    DATA_5    DATA_6  \
 DATA_0  1.000000  0.193894 -0.158920 -0.201927  0.268751  0.101432 -0.135577   
 DATA_1  0.193894  1.000000  0.174979  0.198547 -0.079036  0.192065 -0.274579   
 DATA_2 -0.158920  0.174979  1.000000  0.101374  0.312216 -0.125120  0.274090   
 DATA_3 -0.201927  0.198547  0.101374  1.000000  0.105096  0.570873 -0.028242   
 DATA_4  0.268751 -0.079036  0.312216  0.105096  1.000000  0.323663  0.141714   
 DATA_5  0.101432  0.192065 -0.125120  0.570873  0.323663  1.000000  0.011200   
 D

In [ ]:
# Converter os campos 'DATA_X' para valores numéricos para análise de correlação
data_numeric = data[['DATA_0', 'DATA_1', 'DATA_2', 'DATA_3',
                     'DATA_4', 'DATA_5', 'DATA_6', 'DATA_7']].apply(lambda x: x.apply(int, base=16))

# Adicionar a flag como numérica (T = 1, R = 0) para facilitar a análise
data_numeric['Flag'] = data['Flag'].apply(lambda x: 1 if x == 'T' else 0)

# Calcular a correlação entre os campos de dados e a flag
correlation_with_flag = data_numeric.corr()['Flag'].drop('Flag')

correlation_with_flag


,Flag
DATA_0,-0.398449
DATA_1,-0.483304
DATA_2,-0.396782
DATA_3,-0.418723
DATA_4,-0.399936
DATA_5,-0.489275
DATA_6,-0.296161
DATA_7,-0.413866


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando o modelo de Regressão Logística
model_minmax = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    max_iter=500,
    random_state=42
)

# Treinando o modelo com os dados de treino
model_minmax.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliando no conjunto de treino
y_train_pred = model_minmax.predict(X_train_scaled)
y_train_prob = model_minmax.predict_proba(X_train_scaled)[:, 1]
print_metrics(y_train, y_train_pred, y_train_prob, "Treino")


# Avaliando no conjunto de teste
y_test_pred = model_minmax.predict(X_test_scaled)
y_test_prob = model_minmax.predict_proba(X_test_scaled)[:, 1]
print_metrics(y_test, y_test_pred, y_test_prob, "Teste")

# Avaliando no conjunto de validação
y_val_pred = model_minmax.predict(X_val_scaled)
y_val_prob = model_minmax.predict_proba(X_val_scaled)[:, 1]
print_metrics(y_val, y_val_pred, y_val_prob, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 0.9274
Precisão: 0.8611
Recall: 1.0000
F1-Score: 0.9254
AUC-ROC: 0.9842

Matriz de Confusão:
[[2578  392]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9322
Precisão: 0.8667
Recall: 1.0000
F1-Score: 0.9286
AUC-ROC: 0.9881

Matriz de Confusão:
[[885 122]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9283
Precisão: 0.8576
Recall: 1.0000
F1-Score: 0.9234
AUC-ROC: 0.9844

Matriz de Confusão:
[[894 129]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo Random Forest com regularização
model_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_rf(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_rf = model_rf.predict(X_train_scaled)
y_train_prob_rf = model_rf.predict_proba(X_train_scaled)[:, 1]
print_metrics_rf(y_train, y_train_pred_rf, y_train_prob_rf, "Treino")


# Avaliar no conjunto de teste
y_test_pred_rf = model_rf.predict(X_test_scaled)
y_test_prob_rf = model_rf.predict_proba(X_test_scaled)[:, 1]
print_metrics_rf(y_test, y_test_pred_rf, y_test_prob_rf, "Teste")

# Avaliar no conjunto de validação
y_val_pred_rf = model_rf.predict(X_val_scaled)
y_val_prob_rf = model_rf.predict_proba(X_val_scaled)[:, 1]
print_metrics_rf(y_val, y_val_pred_rf, y_val_prob_rf, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo de Árvore de Decisão com regularização
model_tree = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model_tree.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_tree(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_tree = model_tree.predict(X_train_scaled)
y_train_prob_tree = model_tree.predict_proba(X_train_scaled)[:, 1]
print_metrics_tree(y_train, y_train_pred_tree, y_train_prob_tree, "Treino")


# Avaliar no conjunto de teste
y_test_pred_tree = model_tree.predict(X_test_scaled)
y_test_prob_tree = model_tree.predict_proba(X_test_scaled)[:, 1]
print_metrics_tree(y_test, y_test_pred_tree, y_test_prob_tree, "Teste")

# Avaliar no conjunto de validação
y_val_pred_tree = model_tree.predict(X_val_scaled)
y_val_prob_tree = model_tree.predict_proba(X_val_scaled)[:, 1]
print_metrics_tree(y_val, y_val_pred_tree, y_val_prob_tree, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo de Gradient Boosting
model_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model_gb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_gb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

# Avaliar no conjunto de treino
y_train_pred_gb = model_gb.predict(X_train_scaled)
y_train_prob_gb = model_gb.predict_proba(X_train_scaled)[:, 1]
print_metrics_gb(y_train, y_train_pred_gb, y_train_prob_gb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_gb = model_gb.predict(X_test_scaled)
y_test_prob_gb = model_gb.predict_proba(X_test_scaled)[:, 1]
print_metrics_gb(y_test, y_test_pred_gb, y_test_prob_gb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_gb = model_gb.predict(X_val_scaled)
y_val_prob_gb = model_gb.predict_proba(X_val_scaled)[:, 1]
print_metrics_gb(y_val, y_val_pred_gb, y_val_prob_gb, "Validação")


Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo K-Nearest Neighbors
model_knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    metric='minkowski',
    n_jobs=-1
)

model_knn.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_knn(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_knn = model_knn.predict(X_train_scaled)
y_train_prob_knn = model_knn.predict_proba(X_train_scaled)[:, 1]
print_metrics_knn(y_train, y_train_pred_knn, y_train_prob_knn, "Treino")


# Avaliar no conjunto de teste
y_test_pred_knn = model_knn.predict(X_test_scaled)
y_test_prob_knn = model_knn.predict_proba(X_test_scaled)[:, 1]
print_metrics_knn(y_test, y_test_pred_knn, y_test_prob_knn, "Teste")

# Avaliar no conjunto de validação
y_val_pred_knn = model_knn.predict(X_val_scaled)
y_val_prob_knn = model_knn.predict_proba(X_val_scaled)[:, 1]
print_metrics_knn(y_val, y_val_pred_knn, y_val_prob_knn, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]


In [ ]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo SVM
model_svm = SVC(
    kernel='rbf',
    C=1.0,
    probability=True,
    random_state=42
)

model_svm.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_svm(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

# Avaliar no conjunto de treino
y_train_pred_svm = model_svm.predict(X_train_scaled)
y_train_prob_svm = model_svm.predict_proba(X_train_scaled)[:, 1]
print_metrics_svm(y_train, y_train_pred_svm, y_train_prob_svm, "Treino")


# Avaliar no conjunto de teste
y_test_pred_svm = model_svm.predict(X_test_scaled)
y_test_prob_svm = model_svm.predict_proba(X_test_scaled)[:, 1]
print_metrics_svm(y_test, y_test_pred_svm, y_test_prob_svm, "Teste")

# Avaliar no conjunto de validação
y_val_pred_svm = model_svm.predict(X_val_scaled)
y_val_prob_svm = model_svm.predict_proba(X_val_scaled)[:, 1]
print_metrics_svm(y_val, y_val_pred_svm, y_val_prob_svm, "Validação")




Métricas para o conjunto de Treino:
Acurácia: 0.9657
Precisão: 0.9293
Recall: 1.0000
F1-Score: 0.9633
AUC-ROC: 0.9842

Matriz de Confusão:
[[2785  185]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9667
Precisão: 0.9297
Recall: 1.0000
F1-Score: 0.9635
AUC-ROC: 0.9881

Matriz de Confusão:
[[947  60]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9678
Precisão: 0.9305
Recall: 1.0000
F1-Score: 0.9640
AUC-ROC: 0.9844

Matriz de Confusão:
[[965  58]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo XGBoost
model_xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=10,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model_xgb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_xgb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_xgb = model_xgb.predict(X_train_scaled)
y_train_prob_xgb = model_xgb.predict_proba(X_train_scaled)[:, 1]
print_metrics_xgb(y_train, y_train_pred_xgb, y_train_prob_xgb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_xgb = model_xgb.predict(X_test_scaled)
y_test_prob_xgb = model_xgb.predict_proba(X_test_scaled)[:, 1]
print_metrics_xgb(y_test, y_test_pred_xgb, y_test_prob_xgb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_xgb = model_xgb.predict(X_val_scaled)
y_val_prob_xgb = model_xgb.predict_proba(X_val_scaled)[:, 1]
print_metrics_xgb(y_val, y_val_pred_xgb, y_val_prob_xgb, "Validação")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:32:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, roc_auc_score, f1_score
)

# Supondo que 'data_numeric' esteja carregado corretamente
X = data_numeric.drop('Flag', axis=1)
y = data_numeric['Flag']

# Dividindo os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicando o MinMaxScaler nos conjuntos de treino, validação e teste
scaler_minmax = MinMaxScaler()

X_train_scaled = scaler_minmax.fit_transform(X_train)
X_val_scaled = scaler_minmax.transform(X_val)
X_test_scaled = scaler_minmax.transform(X_test)

# Criando e treinando o modelo CatBoost
model_cb = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=10,
    random_seed=42,
    verbose=0
)

model_cb.fit(X_train_scaled, y_train)

# Função para calcular e exibir métricas
def print_metrics_cb(y_true, y_pred, y_prob, dataset_name):
    print(f"\nMétricas para o conjunto de {dataset_name}:")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, pos_label=1):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_true, y_prob):.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))


# Avaliar no conjunto de treino
y_train_pred_cb = model_cb.predict(X_train_scaled)
y_train_prob_cb = model_cb.predict_proba(X_train_scaled)[:, 1]
print_metrics_cb(y_train, y_train_pred_cb, y_train_prob_cb, "Treino")


# Avaliar no conjunto de teste
y_test_pred_cb = model_cb.predict(X_test_scaled)
y_test_prob_cb = model_cb.predict_proba(X_test_scaled)[:, 1]
print_metrics_cb(y_test, y_test_pred_cb, y_test_prob_cb, "Teste")

# Avaliar no conjunto de validação
y_val_pred_cb = model_cb.predict(X_val_scaled)
y_val_prob_cb = model_cb.predict_proba(X_val_scaled)[:, 1]
print_metrics_cb(y_val, y_val_pred_cb, y_val_prob_cb, "Validação")



Métricas para o conjunto de Treino:
Acurácia: 0.9826
Precisão: 0.9628
Recall: 1.0000
F1-Score: 0.9810
AUC-ROC: 0.9842

Matriz de Confusão:
[[2876   94]
 [   0 2430]]

Métricas para o conjunto de Teste:
Acurácia: 0.9867
Precisão: 0.9706
Recall: 1.0000
F1-Score: 0.9851
AUC-ROC: 0.9881

Matriz de Confusão:
[[983  24]
 [  0 793]]

Métricas para o conjunto de Validação:
Acurácia: 0.9822
Precisão: 0.9604
Recall: 1.0000
F1-Score: 0.9798
AUC-ROC: 0.9844

Matriz de Confusão:
[[991  32]
 [  0 777]]
